# TSP

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have proto

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.linear_solver import pywraplp

In [ ]:
def TSP(df):
    list_2d = df.values.tolist()
    def create_data_model():
         data = {}
         data['distance_matrix'] = list_2d
         data['num_vehicles'] = 1
         data['depot'] = 0
         return data
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
      routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    def print_solution(manager, routing, solution):
        """Prints solution on console."""
        print('Objective: {} m'.format(solution.ObjectiveValue()))
        index = routing.Start(0)
        plan_output = 'Route for vehicle 0:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        print(plan_output)
        plan_output += 'Route distance: {}m\n'.format(route_distance)
    return print_solution(manager, routing, solution)

In [ ]:
df_monday = pd.read_excel('Matrix.xlsx',sheet_name='Monday',index_col=0)
print("Monday:")
TSP(df_monday)
df_tuesday = pd.read_excel('Matrix.xlsx',sheet_name='Tuesday',index_col=0)
print("Tuesday:")
TSP(df_tuesday)
print("Wednesday:")
df_wednesday = pd.read_excel('Matrix.xlsx',sheet_name='Wednesday',index_col=0)
TSP(df_wednesday)
print("Thursday:")
df_thursday = pd.read_excel('Matrix.xlsx',sheet_name='Thursday',index_col=0)
TSP(df_thursday)
print("Friday:")
df_friday = pd.read_excel('Matrix.xlsx',sheet_name='Friday',index_col=0)
TSP(df_friday)
print("Saturday:")
#df_saturday = pd.read_excel('Matrix.xlsx',sheet_name='Saturday',index_col=0)
#TSP(df_saturday)

Monday:
Objective: 12950 m
Route for vehicle 0:
 0 -> 1 -> 2 -> 5 -> 4 -> 3 -> 6 -> 0

Tuesday:
Objective: 19400 m
Route for vehicle 0:
 0 -> 2 -> 4 -> 6 -> 1 -> 5 -> 3 -> 0

Wednesday:
Objective: 36170 m
Route for vehicle 0:
 0 -> 17 -> 3 -> 15 -> 16 -> 10 -> 9 -> 20 -> 13 -> 14 -> 12 -> 19 -> 11 -> 8 -> 18 -> 4 -> 5 -> 22 -> 7 -> 6 -> 2 -> 23 -> 24 -> 1 -> 21 -> 0

Thursday:
Objective: 16750 m
Route for vehicle 0:
 0 -> 6 -> 3 -> 4 -> 5 -> 2 -> 7 -> 1 -> 0

Friday:
Objective: 21950 m
Route for vehicle 0:
 0 -> 5 -> 3 -> 1 -> 2 -> 4 -> 0

Saturday:
